In [ ]:
import pickle
import random

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy as sc
from scipy.spatial import Delaunay

from simplicial_kuramoto import SimplicialComplex, plotting
from simplicial_kuramoto.graph_generator import modular_graph
from simplicial_kuramoto.integrators import *

In [ ]:
def get_grid(n_node):

    x = np.linspace(0, 1, n_node)

    points = []
    for i in range(n_node):
        for j in range(n_node):
            points.append([x[j], x[i]])

    tri = Delaunay(points)

    edge_list = []
    for t in tri.simplices:
        edge_list.append([t[0], t[1]])
        edge_list.append([t[0], t[2]])
        edge_list.append([t[1], t[2]])

    graph = nx.Graph()
    graph.add_nodes_from(np.arange(len(points)))
    graph.add_edges_from(edge_list)
    return graph, points

In [ ]:
graph, points = get_grid(5)

plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(graph, pos=points, node_size=20)
nx.draw_networkx_edges(graph, pos=points)

labels = dict(((u, v), d) for d, (u, v) in enumerate(graph.edges))
nx.draw_networkx_edge_labels(graph, pos=points, edge_labels=labels)

In [ ]:
initial_phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))

edge_flip = 32
initial_phase = np.zeros(len(graph.edges))
initial_phase[edge_flip] = np.pi

t_max = 100
n_t = 100

complex_delaunay = SimplicialComplex(graph=graph, no_faces=True)

edge_result = integrate_edge_kuramoto(complex_delaunay, initial_phase, t_max, n_t)


plotting.plot_edge_kuramoto(edge_result)

plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    graph,
    pos=points,
    edge_color=edge_result.y[:, -1],
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
    edge_vmin=np.min(edge_result.y),
    edge_vmax=np.max(edge_result.y),
)

nx.draw_networkx_edge_labels(graph, pos=points, edge_labels=labels)

op = plotting.plot_order_parameter(edge_result.y, return_op=True, plot=False)
print("Order parameter: {}".format(op))

In [ ]:
complex_delaunay = SimplicialComplex(graph=graph, no_faces=False)

edge_result = integrate_edge_kuramoto(complex_delaunay, initial_phase, t_max, n_t)


plotting.plot_edge_kuramoto(edge_result)


plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    graph,
    pos=points,
    edge_color=edge_result.y[:, -1],
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
    edge_vmin=np.min(edge_result.y),
    edge_vmax=np.max(edge_result.y),
)

nx.draw_networkx_edge_labels(graph, pos=points, edge_labels=labels)

op = plotting.plot_order_parameter(edge_result.y, return_op=True, plot=False)
print("Order parameter: {}".format(op[-1]))

In [ ]:
B0 = complex_delaunay.node_incidence_matrix
W0 = complex_delaunay.node_weights_matrix
B1 = complex_delaunay.edge_incidence_matrix
W1 = complex_delaunay.edge_weights_matrix
W2 = complex_delaunay.face_weights_matrix
omega_0 = np.zeros(complex_delaunay.n_edges)

Ld = W1.dot(B0.dot(W0.dot(B0.T)))
Lu = W1.dot(B1.T.dot(W2.dot(B1)))

L1 = Ld + Lu

L1.dot(edge_result.y[:, -1])

# Increasing number of faces

In [ ]:
initial_phase = np.zeros(len(graph.edges))

# randomly flip 10 edges
for j in range(10):
    initial_phase[np.random.randint(1, len(graph.edges))] = np.pi


all_cliques = nx.enumerate_all_cliques(graph)
faces = [clique for clique in all_cliques if len(clique) == 3]

order = []
n_repeats = 10
for i in np.linspace(0, 30, 31):
    order_repeat = []
    for k in range(0, n_repeats):
        rand_faces = random.sample(faces, int(i))
        complex_delaunay = SimplicialComplex(
            graph=graph, no_faces=False, faces=rand_faces
        )
        edge_result = integrate_edge_kuramoto(
            complex_delaunay, initial_phase, t_max, n_t
        )

        op = plotting.plot_order_parameter(edge_result.y, return_op=True, plot=False)
        order_repeat.append(op[-1])

    order.append(order_repeat)

In [ ]:
plt.plot(order)

# Increasing number of flipped edges 

In [ ]:
n_repeats = 10
order = []
for i in range(len(graph.edges)):
    order_repeat = []
    for k in range(n_repeats):
        initial_phase = np.zeros(len(graph.edges))

        for j in range(i):
            initial_phase[np.random.randint(1, len(graph.edges))] = np.pi

        complex_delaunay = SimplicialComplex(graph=graph, no_faces=False)
        edge_result = integrate_edge_kuramoto(
            complex_delaunay, initial_phase, t_max, n_t
        )
        op = plotting.plot_order_parameter(edge_result.y, return_op=True, plot=False)
        print("Order parameter: {}".format(op[-1]))
        order_repeat.append(op)

    order.append(order_repeat)

In [ ]:
mean_order = np.array([np.mean(l) for l in order])
std_order = np.array([np.std(l) for l in order])

fig, ax = plt.subplots(1)
ax.plot(range(len(graph.edges)), mean_order, lw=2, label="Mean Order", color="blue")
ax.fill_between(
    range(len(graph.edges)),
    mean_order + std_order,
    mean_order - std_order,
    facecolor="blue",
    alpha=0.5,
)
ax.set_title("Increasing the number of flipped edges")
ax.legend(loc="upper right")
ax.set_xlabel("No. flipped edges")
ax.set_ylabel("Order")
ax.grid()

In [ ]:
edge_result = integrate_edge_kuramoto(complex_delaunay, initial_phase, t_max, n_t)


plotting.plot_edge_kuramoto(edge_result)


plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    graph,
    pos=points,
    edge_color=edge_result.y[:, -1],
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
    edge_vmin=np.min(edge_result.y),
    edge_vmax=np.max(edge_result.y),
)

nx.draw_networkx_edge_labels(graph, pos=points, edge_labels=labels)

op = plotting.plot_order_parameter(edge_result.y, return_op=True, plot=False)
print("Order parameter: {}".format(op[-1]))

# Creating densities of faces


In [ ]:
# adding faces at the centre of the grid

graph, points = get_grid(5)

plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(graph, pos=points, node_size=20)
nx.draw_networkx_edges(graph, pos=points)

all_cliques = nx.enumerate_all_cliques(graph)
faces = [clique for clique in all_cliques if len(clique) == 3]

face_coords = np.array(
    [
        np.mean(np.vstack([points[node] for node in face]), axis=0).tolist()
        for face in faces
    ]
)
face_distance_centre = np.sqrt(((face_coords - 0.5) ** 2).sum(axis=1))
order_distances = np.argsort(face_distance_centre)  # reverse order


n_repeats = 10
n_flips = np.linspace(1, 10, 10).astype(np.int64)

order = np.zeros([n_repeats, len(order_distances), len(n_flips)])

for f, n_flipped in enumerate(n_flips):
    for n in range(n_repeats):

        initial_phase = np.zeros(len(graph.edges))
        for k in range(n_flipped):
            initial_phase[np.random.randint(1, len(graph.edges))] = np.pi

        t_max = 100
        n_t = 100

        # loop over the faces
        for i in range(len(order_distances)):
            top_faces_idx = order_distances[:i]
            faces_to_add = [faces[j] for j in top_faces_idx]

            if faces_to_add:
                complex_delaunay = SimplicialComplex(
                    graph=graph, no_faces=False, faces=faces_to_add
                )
            else:
                complex_delaunay = SimplicialComplex(graph=graph, no_faces=True)

            edge_result = integrate_edge_kuramoto(
                complex_delaunay, initial_phase, t_max, n_t
            )

            op = plotting.plot_order_parameter(
                edge_result.y, return_op=True, plot=False
            )
            order[n, i, f] = op[-1]

filehandler = open("kuramoto_order_centre_faces.obj", "wb")
pickle.dump(order, filehandler)
filehandler.close()

In [ ]:
for k in range(10):
    mean_order = order[:, :, k].mean(axis=0)
    std_order = order[:, :, k].std(axis=0)

    fig, ax = plt.subplots(1)
    ax.plot(range(order.shape[1]), mean_order, lw=2, label="Mean Order", color="blue")
    ax.fill_between(
        range(order.shape[1]),
        mean_order + std_order,
        mean_order - std_order,
        facecolor="blue",
        alpha=0.5,
    )
    ax.set_title(
        "Increasing the number of faces from centre (flipped edges: {})".format(k + 1)
    )
    ax.legend(loc="upper right")
    ax.set_xlabel("No. faces")
    ax.set_ylabel("Order")
    ax.grid()

In [ ]:
# adding faces at the edge of the grid


graph, points = get_grid(5)

plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(graph, pos=points, node_size=20)
nx.draw_networkx_edges(graph, pos=points)

all_cliques = nx.enumerate_all_cliques(graph)
faces = [clique for clique in all_cliques if len(clique) == 3]

face_coords = np.array(
    [
        np.mean(np.vstack([points[node] for node in face]), axis=0).tolist()
        for face in faces
    ]
)
face_distance_centre = np.sqrt(((face_coords - 0.5) ** 2).sum(axis=1))
order_distances = np.argsort(face_distance_centre)[::-1]  # reverse order


n_repeats = 10
n_flips = np.linspace(1, 10, 10).astype(np.int64)

order_r = np.zeros([n_repeats, len(order_distances), len(n_flips)])

for f, n_flipped in enumerate(n_flips):
    for n in range(n_repeats):

        initial_phase = np.zeros(len(graph.edges))
        for k in range(n_flipped):
            initial_phase[np.random.randint(1, len(graph.edges))] = np.pi

        t_max = 100
        n_t = 100

        # loop over the faces
        for i in range(len(order_distances)):
            top_faces_idx = order_distances[:i]
            faces_to_add = [faces[j] for j in top_faces_idx]

            if faces_to_add:
                complex_delaunay = SimplicialComplex(
                    graph=graph, no_faces=False, faces=faces_to_add
                )
            else:
                complex_delaunay = SimplicialComplex(graph=graph, no_faces=True)

            edge_result = integrate_edge_kuramoto(
                complex_delaunay, initial_phase, t_max, n_t
            )

            op = plotting.plot_order_parameter(
                edge_result.y, return_op=True, plot=False
            )
            order_r[n, i, f] = op[-1]

filehandler = open("kuramoto_order_edge_faces.obj", "wb")
pickle.dump(order_r, filehandler)
filehandler.close()

In [ ]:
for k in range(10):
    mean_order = order_r[:, :, k].mean(axis=0)
    std_order = order_r[:, :, k].std(axis=0)

    fig, ax = plt.subplots(1)
    ax.plot(range(order_r.shape[1]), mean_order, lw=2, label="Mean Order", color="blue")
    ax.fill_between(
        range(order_r.shape[1]),
        mean_order + std_order,
        mean_order - std_order,
        facecolor="blue",
        alpha=0.5,
    )
    ax.set_title(
        "Increasing the number of faces from edge of graph (flipped edges: {})".format(
            k + 1
        )
    )
    ax.legend(loc="upper right")
    ax.set_xlabel("No. faces")
    ax.set_ylabel("Order")
    ax.grid()

# confirm convergence